## Authenticate to Google Earth Engine

In [1]:
import ee
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

In [2]:
def get_feature_collection(asset_id):
    """Check if an asset exists, and load it as a FeatureCollection if it does.
    Otherwise, return an empty FeatureCollection.
    
    Args:
        asset_id (str): The Earth Engine asset ID.
        
    Returns:
        ee.FeatureCollection: The loaded FeatureCollection or an empty one.
    """
    try:
        # Get asset information to check existence
        ee.data.getAsset(asset_id)
        print(f"Asset '{asset_id}' exists. Loading FeatureCollection.")
        return ee.FeatureCollection(asset_id)
    except Exception as e:
        print(f"Asset '{asset_id}' does not exist. Returning empty FeatureCollection.")
        return ee.FeatureCollection([])

In [3]:
filename_prefix = "Area_bheramgarh"
suffix = filename_prefix.split("_")[-1]
mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

def get_class_and_mapping(vectors, vector_name):
    vector_of_class = vectors.filter(ee.Filter.eq('class', vector_name))
    vector_of_class_with_map = vector_of_class.map(lambda f: f.set('class', mapping[vector_name]))
    return vector_of_class, vector_of_class_with_map

all = get_feature_collection("projects/ee-raman/assets/all_" + suffix)
farm, farm_with_map = get_class_and_mapping(all, "farm")
scrubland, scrubland_with_map = get_class_and_mapping(all, "scrubland")
plantation, plantation_with_map = get_class_and_mapping(all, "plantation")
rest, rest_with_map = get_class_and_mapping(all, "rest")

ts_data = ee.Image("projects/ee-raman/assets/ts_data_" + suffix)
training_features = farm_with_map.merge(scrubland_with_map).merge(plantation_with_map)


Asset 'projects/ee-raman/assets/all_bheramgarh' exists. Loading FeatureCollection.


In [4]:
import math
from itertools import product


# Function to convert latitude to pixel Y at a given zoom level
def lat_to_pixel_y(lat, zoom):
    sin_lat = math.sin(math.radians(lat))
    pixel_y = ((0.5 - math.log((1 + sin_lat) / (1 - sin_lat)) / (4 * math.pi)) * (2 ** (zoom + 8)))
    return pixel_y

# Function to convert longitude to pixel X at a given zoom level
def lon_to_pixel_x(lon, zoom):
    pixel_x = ((lon + 180) / 360) * (2 ** (zoom + 8))
    return pixel_x

# Function to convert pixel X to longitude
def pixel_x_to_lon(pixel_x, zoom):
    lon = (pixel_x / (2 ** (zoom + 8))) * 360 - 180
    return lon

# Function to convert pixel Y to latitude
def pixel_y_to_lat(pixel_y, zoom):
    n = math.pi - 2 * math.pi * pixel_y / (2 ** (zoom + 8))
    lat = math.degrees(math.atan(math.sinh(n)))
    return lat

def lat_lon_from_pixel(lat, lon, zoom, scale):
    """
    Given a starting latitude and longitude, calculate the latitude and longitude
    of the opposite corner of a 256x256 image at a given zoom level.
    """
    pixel_x = lon_to_pixel_x(lon, zoom)
    pixel_y = lat_to_pixel_y(lat, zoom)
    
    new_lon = pixel_x_to_lon(pixel_x + 256*scale, zoom)
    new_lat = pixel_y_to_lat(pixel_y + 256*scale, zoom)

    return new_lat, new_lon

"""

Helper function for dividing an roi into blocks

"""
def get_n_boxes(lat, lon, n, zoom, scale):
    diagonal_lat_lon = [(lat, lon),]
    for i in range(n):
        new_lat_lon = lat_lon_from_pixel(lat, lon, zoom, scale)
        diagonal_lat_lon.append(new_lat_lon)
        lat, lon = new_lat_lon
    lats = [i[0] for i in diagonal_lat_lon]
    longs = [i[1] for i in diagonal_lat_lon]
    return list(product(lats, longs))

def get_points(roi):
    zoom = 17
    scale = 16
    bounds = roi.bounds().coordinates().get(0).getInfo()
    lons = sorted([i[0] for i in bounds])
    lats = sorted([i[1] for i in bounds])
    starting_point = lats[-1], lons[0]
    min_, max_ = (
        [lon_to_pixel_x(lons[0], zoom), lat_to_pixel_y(lats[0], zoom) ],
        [lon_to_pixel_x(lons[-1], zoom), lat_to_pixel_y(lats[-1], zoom)]
        )
    iterations = math.ceil(max(abs(min_[0] -  max_[0]), abs(min_[1] - max_[1]))/256/16)
    points = get_n_boxes(starting_point[0], starting_point[1], iterations, zoom, scale)
    intersect_list = []
    print(len(points))
    for point in points:
        top_left = point
        bottom_right = lat_lon_from_pixel(top_left[0], top_left[1], zoom, scale)
        rectangle = ee.Geometry.Rectangle([(top_left[1], top_left[0]), (bottom_right[1], bottom_right[0])])
        print(top_left, bottom_right)
        intersects = roi.geometry().intersects(rectangle, ee.ErrorMargin(1)).getInfo()
        if intersects:
            intersect_list.append([top_left, bottom_right])
        print(intersects)
    return intersect_list

roi_boundary = ee.FeatureCollection("users/mtpictd/india_block_boundaries").filter(ee.Filter.eq("block", "Bheramgarh"))
filename_prefix = "Area_bheramgarh"

points = get_points(roi_boundary)

roi_boundary = ee.FeatureCollection([ee.Feature(ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])) for top_left, bottom_right in points])


256
(19.35210359962618, 80.60491122661094) (19.31063593149325, 80.64885653911097)
False
(19.35210359962618, 80.64885653911097) (19.31063593149325, 80.69280185161097)
False
(19.35210359962618, 80.69280185161097) (19.31063593149325, 80.73674716411097)
False
(19.35210359962618, 80.73674716411097) (19.31063593149325, 80.78069247661097)
True
(19.35210359962618, 80.78069247661097) (19.31063593149325, 80.82463778911097)
True
(19.35210359962618, 80.82463778911097) (19.31063593149325, 80.86858310161097)
True
(19.35210359962618, 80.86858310161097) (19.31063593149325, 80.91252841411097)
True
(19.35210359962618, 80.91252841411097) (19.31063593149325, 80.95647372661097)
True
(19.35210359962618, 80.95647372661097) (19.31063593149325, 81.00041903911097)
True
(19.35210359962618, 81.00041903911097) (19.31063593149325, 81.04436435161097)
False
(19.35210359962618, 81.04436435161097) (19.31063593149325, 81.08830966411097)
False
(19.35210359962618, 81.08830966411097) (19.31063593149325, 81.13225497661097)


In [40]:

# Define the bounding box
bounds = roi_boundary.geometry().bounds()

# Define grid size
grid_size = 10  # 10x10
classes = [(1,5), (2,5), (3,3)]

# Create grid function
def create_grid(bounds, n_cols, n_rows):
    coords = bounds.coordinates().get(0)
    coords_list = ee.List(coords)
    xmin = ee.Number(ee.List(coords_list.get(0)).get(0))
    ymin = ee.Number(ee.List(coords_list.get(0)).get(1))
    xmax = ee.Number(ee.List(coords_list.get(2)).get(0))
    ymax = ee.Number(ee.List(coords_list.get(2)).get(1))

    dx = xmax.subtract(xmin).divide(n_cols)
    dy = ymax.subtract(ymin).divide(n_rows)

    cells = []

    for i in range(n_cols):
        for j in range(n_rows):
            x0 = xmin.add(dx.multiply(i))
            y0 = ymin.add(dy.multiply(j))
            x1 = x0.add(dx)
            y1 = y0.add(dy)
            cell = ee.Geometry.Rectangle([x0, y0, x1, y1])
            cells.append(ee.Feature(cell))

    return ee.FeatureCollection(cells)

# Create grid
grid = create_grid(bounds, grid_size, grid_size)

# Function to sample up to 5 features per class from a cell
def sample_from_cell(cell):
    cell_geom = ee.Feature(cell).geometry()

    def get_class_samples(class_name, limit):
        return training_features.filterBounds(cell_geom)\
            .filter(ee.Filter.eq('class', class_name))\
                .limit(limit)

    samples = ee.FeatureCollection([])
    for class_name, limit in classes:
        class_samples = get_class_samples(class_name, limit)
        samples = samples.merge(class_samples)

    return samples

# Map over grid
samples_fc = grid.map(lambda cell: sample_from_cell(cell)).flatten()

In [41]:
# Map over grid
samples_fc = grid.map(lambda cell: sample_from_cell(cell)).flatten()

In [46]:
samples_fc.size().getInfo()

723

In [61]:
points = ee.FeatureCollection.randomPoints(region=samples_fc.geometry(), points=5, seed=42)
    
# Optionally, attach the parent feature's properties to each sampled point
points_with_props = points.map(lambda pt: pt.set(samples_fc.toDictionary()))

In [62]:
points_with_props.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [81.11508458939517, 19.212567745462547]},
   'id': '0',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [81.18825444867296, 19.190675899034797]},
   'id': '1',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [81.11951452017934, 19.047409064267757]},
   'id': '2',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [81.09834956193036, 19.217687187993295]},
   'id': '3',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [80.76888387289696, 19.33769241745884]},
   'id': '4',
   'properties': {}}]}

In [44]:
# Function to sample 5 random points from each feature
def sample_points(feature):
    # Generate 5 random points within the feature's geometry
    points = ee.FeatureCollection.randomPoints(region=feature.geometry(), points=1, seed=42)
    
    # Optionally, attach the parent feature's properties to each sampled point
    points_with_props = points.map(lambda pt: pt.set(feature.toDictionary()))
    return points_with_props

# Map over the feature collection
sampled_points_fc = samples_fc.map(sample_points).flatten()

In [45]:
sampled_points_fc.size().getInfo()

EEException: Unknown Error.

In [20]:
samples_fc.geometry().area().getInfo()/1e6

49.7276514175492

In [9]:
training_data = ts_data.sampleRegions(
    collection=samples_fc,
    properties=['class'],
    scale=10  # Adjust scale based on resolution
)
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training_data,
    classProperty='class',
    inputProperties=ts_data.bandNames()
)

In [10]:
classifier.getInfo()

EEException: Computation timed out.

In [10]:

training_data = ts_data.sampleRegions(
    collection=samples_fc,
    properties=['class'],
    scale=10  # Adjust scale based on resolution
)
classifier = ee.Classifier.smileRandomForest(50).train(
    features=training_data,
    classProperty='class',
    inputProperties=ts_data.bandNames()
)

classified = ts_data.classify(classifier)


In [11]:
task = ee.batch.Export.image.toAsset(
    image=classified.clip(roi_boundary.geometry()),
    description='Classified' + filename_prefix.split("_")[-1],
    assetId='projects/ee-raman/assets/classified_' + filename_prefix.split("_")[-1],
    #pyramidingPolicy = {'predicted_label': 'mode'},
    scale = 10,
    maxPixels = 1e13,
    crs = 'EPSG:4326'
)
task.start()


In [ ]:
def assign_mode_label(feature):
    class_values = classified.reduceRegion(
        reducer=ee.Reducer.mode(),
        geometry=feature.geometry(),
        scale=30,  # Adjust scale as per resolution
        bestEffort=True
    )
    return feature.set('class', class_values.get('classification'))

# Apply function to test features
rest_with_labels = rest_with_map.map(assign_mode_label).filter(ee.Filter.notNull(['class']))
rest_with_labels = rest_with_labels.map(lambda x: x.set('class', reversed_ee_mapping.get(ee.Number(x.get("class")).int())))
all_with_labels = rest_with_labels.merge(farm).merge(scrubland)

In [15]:
task = ee.batch.Export.table.toAsset(
    collection=all_with_labels,
    description='Classification',
    assetId="projects/ee-raman/assets/refined_all_" + suffix
)

# Start the task
task.start()

In [9]:
all_with_labels.size().getInfo()

EEException: Computation timed out.

In [9]:
def add_property_count(feature):
    return feature.set('num_properties', feature.geometry().area())

fc_with_counts = rest_with_map.map(add_property_count)

# Get the maximum property count
max_prop_count = fc_with_counts.aggregate_max('num_properties')

In [10]:
max_prop_count.getInfo()

126930.1696973555

In [11]:
126930/1e6

0.12693